# Python - Final Project
Author: Sebastian Wegerer<br>
MatrNr: 01616960

### Project:
Data Structures and Data Management - Project

People burried at St. Marx cemetery. 

### Content:

1) Import source(s)
2) Search for matches
3) GeoCoder
4) Sort csv(s)



## 1) Import source(s)


CSV files contains several people buried at St_Marx. Different sources could contain bits of information. The idea of this notebook is to create a tool to process and manage the data automatically. 

First source: austriasites
One big block of text copied to a txt file.

In [1]:
import re
import pandas as pd 

#source:
#https://www.austriasites.com/vienna/bezirk03_friedhof_st_marx_graeber.htm

source_austriasites = ''

with open('austriasites.txt') as l_reader:
    source_austriasites = l_reader.read()



Creates a cleaned version of the original csv file. 

In [2]:
#open("St_Marxer_Spreadsheet.csv", "r")

v_df_people = pd.DataFrame(pd.read_csv("St_Marxer_Spreadsheet.csv", sep=";"))

v_df_people.to_csv("St_Marxer_Spreadsheet_Cleaned.csv", index=False)

## 2) Search for matches

A loop searches for every single name from the list in the txt file from above. 

Names found are added to a data frame and saved as csv file.

In [3]:
v_df_people = pd.DataFrame(pd.read_csv("St_Marxer_Spreadsheet_Cleaned.csv", sep=","))

v_df_findings = pd.DataFrame()

v_row_counter = 1

for index in v_df_people.index:
    
    l_occurences = re.findall(v_df_people['varchar Name'][index], source_austriasites)
    
    if l_occurences != []:
        
        
        v_df_findings.loc[v_row_counter, 'varchar Name'] = l_occurences[0]
        v_row_counter = v_row_counter + 1     
        
        
# index=False excludes the index we used to keep track of (row_counter)        
v_df_findings.to_csv("Pemmer_Works_Matches.csv", index=False) 


## 2 -a) Find everything

There is more data to collect than the name. Regex is used to search for names but includes everything until a new line is detected. 

In [6]:
v_df_people = pd.DataFrame(pd.read_csv("St_Marxer_Spreadsheet.csv", sep=";"))

v_df_findings = pd.DataFrame()

v_row_counter = 1

for index in v_df_people.index:
    
    l_search_string = v_df_people['varchar Name'][index] + ".*\n"
    
    l_occurences = re.findall(l_search_string, source_austriasites)
    
    if l_occurences != []:
        
        v_df_findings.loc[v_row_counter, 'varchar Data'] = l_occurences[0]
        v_row_counter = v_row_counter + 1     
        
        
# index=False excludes the index we used to keep track of (row_counter)        
v_df_findings.to_csv("Pemmer_Works_Data.csv", index=False) 



### Resulting problem:

Data like this can't be used. Some sort of processing necessary!

<img src="Pemmer_Problem.png"
     style="float: left; margin-right: 10px;" />

First attempt: split everything up.

In [5]:
v_df_data = pd.DataFrame(pd.read_csv("Pemmer_Works_Data.csv", sep=","))

v_df_reworked = pd.DataFrame()


for index in v_df_data.index:
    l_fields = v_df_data.loc[index, 'varchar Data'].split(",")
    
    for element in l_fields:
        v_df_reworked.loc[index, 'column: ' + str(l_fields.index(element))] = element

v_df_reworked.to_csv("Pemmer_Works_Data.csv", index=False)

## column: 0 

The names can be used but everything else is out of order. Slower attempt needed!

<img src="Pemmer_Problem_2.png"
     style="float: left; margin-right: 10px;" />

## <mark>--->>^ execute: 2 a-) Find everything ^<<---</mark>

Separates the names.

In [7]:
v_df_data = pd.DataFrame(pd.read_csv("Pemmer_Works_Data.csv", sep=","))

v_df_reworked = pd.DataFrame()


for index in v_df_data.index:
    l_fields = v_df_data.loc[index, 'varchar Data'].split(",", 1)
    
    v_df_reworked.loc[index, 'varchar Name'] = l_fields[0]
    
    if len(l_fields) > 1:
        v_df_reworked.loc[index, 'varchar Data'] = l_fields[1]

v_df_reworked.to_csv("Pemmer_Works_Data.csv", index=False)

Separates the maiden name.

In [8]:
v_df_data = pd.DataFrame(pd.read_csv("Pemmer_Works_Data.csv", sep=","))

v_df_reworked = v_df_data


for index in v_df_data.index:

    # Regex is not precise but works for the data
    if v_df_data.loc[index].str.contains('geb.', case=True, regex=True)[1] == True:
        l_row = v_df_data.loc[index, 'varchar Data']
        l_row = l_row.split(",", 1)
        
        
        v_df_reworked.loc[index, 'varchar Maiden_Name'] = l_row[0].split(" ")[-1]
        v_df_reworked.loc[index, 'varchar Data'] = l_row[1]
    
    if v_df_data.loc[index].str.contains('\d\d\.\d\d\.\d\d\d\d.*\d\d\.\d\d\.\d\d\d\d', case=True, regex=True)[1] == True:
        l_row = v_df_data.loc[index, 'varchar Data']

v_df_reworked.to_csv("Pemmer_Works_Data.csv", index=False)

Merges the found and prepared data with the original data file.

In [10]:
v_df_data = pd.DataFrame(pd.read_csv("St_Marxer_Spreadsheet_Cleaned.csv", sep=","))
v_df_source = pd.DataFrame(pd.read_csv("Pemmer_Works_Data.csv", sep=","))


v_df_merged = pd.merge(
    v_df_data,
    v_df_source,
    how="left",
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

v_df_merged.to_csv("Merged.csv", index=False)

## 3) GeoCoder

Looks up coordinates for addresses

In [11]:
import geopy.geocoders as gg

# User input. Counts to 10.
v_bool_exit = 0

# In case AddressToCoord is called without a person to attach
v_row_counter = 0

# Empty DataFrame to write unattached coords to
v_df_locations = pd.DataFrame()
v_df_location_attachments = pd.DataFrame(pd.read_csv("Pemmer_Works_Matches.csv", sep=","))


v_geolocator = gg.Nominatim(user_agent="sebastian.wegerer@hotmail.com")
# ---------------------------------------------

# Returs coords for an address. 
# Data can be attached to a person in the main work spreadsheet
def AddressToCoord(p_address, p_person=None):    
    
    l_location = v_geolocator.geocode(p_address)
    l_new_lat = l_location.latitude
    l_new_long = l_location.longitude 
    
    # Creates a row in the DataFrame.
    # DataFrame is printed to csv after user input is closed.
    if p_person == None:
        v_df_locations.loc[v_row_counter, 'lat'] = l_new_lat
        v_df_locations.loc[v_row_counter, 'long'] = l_new_long
        v_df_locations.loc[v_row_counter, 'address'] = p_address
    else:
        # Data has to attached to person
        # Checks for matches.
        if any(v_df_location_attachments.loc[v_df_location_attachments['varchar Name'] == (p_person), 'varchar Name']):
            v_df_location_attachments.loc[v_df_location_attachments['varchar Name'] == (p_person), 'lat'] = l_new_lat
            v_df_location_attachments.loc[v_df_location_attachments['varchar Name'] == (p_person), 'long'] = l_new_long
            v_df_location_attachments.loc[v_df_location_attachments['varchar Name'] == (p_person), 'address'] = p_address
        else:
            print("Warning E05: No one found with the name: " + p_person + ". Coords for: " + p_address + " are not saved!") 
         
        
        
# ----------------------------------------------------------       


while v_bool_exit == 0:
    l_new_address = input("Exit with: -1- or enter the address you would like to check: ")
    
    if l_new_address == "1":
        v_bool_exit = 1
    else:
        l_person = input("Skip with: -2- or enter name to attach: ")
        
        if l_person == "2":
            AddressToCoord(l_new_address)
            v_row_counter = v_row_counter + 1
        else:
            AddressToCoord(l_new_address, l_person)

        
v_df_locations.to_csv("Address_Spreadsheet.csv", index=False)  
v_df_location_attachments.to_csv("Pemmer_Works_Matches.csv", index=False)  




Exit with: -1- or enter the address you would like to check:  Ringstrasse 1
Skip with: -2- or enter name to attach:  Carl Khym
Exit with: -1- or enter the address you would like to check:  Rathausplatz 1
Skip with: -2- or enter name to attach:  Superman


Warning E05: No one found with the name: Superman. Coords for: Rathausplatz 1 are not saved!


Exit with: -1- or enter the address you would like to check:  Rathausplatz 1
Skip with: -2- or enter name to attach:  2
Exit with: -1- or enter the address you would like to check:  1


## 4) Sort csv(s)


Simple Input for file and optional column to sort ascending.

In [18]:
v_bool_exit = 0

while v_bool_exit == 0:
    l_file = input("Exit with: -1- or enter the name of the file you would like to sort: ")
    
    if l_file == "1":
        v_bool_exit = 1
    else:
        v_df_sort = pd.DataFrame(pd.read_csv("Pemmer_Works_Matches.csv", sep=","))

        
        l_column = input("Skip with: -2- or enter column to sort by: ")
        
        if l_column == "2":
            v_df_sorted = v_df_sort.sort_values(by='varchar_Name', ascending=True, na_position='first')

            AddressToCoord(l_file)
        else:
            v_df_sorted = v_df_sort.sort_values(by=l_column, ascending=True, na_position='first')

  
        v_df_sorted.to_csv(l_file + ".csv", index=False)  




Exit with: -1- or enter the name of the file you would like to sort:  Pemmer_Works_Spreadsheet
Skip with: -2- or enter column to sort by:  varchar Name
Exit with: -1- or enter the name of the file you would like to sort:  1
